<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=cdfee68ee04546e8afc446d9fc63e215c313ad1784cefeea844dbb0cc01755d2
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=cf679fb02e1207399e28e66fef9cca91a3ef720d887b5c024953bd68f26b7a23
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # ディレクトリ名作成（YYYYMMDD_screening）
    now = dt.datetime.now().strftime("%Y%m%d")
    directory = f'{now}_screening'

    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 267.92811489105225秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2541800,644.200012
1,ＩＮＰＥＸ,1605,鉱業,15245400,1637.000000
2,安藤・間,1719,建設業,1043300,1059.000000
3,コムシスホールディングス,1721,建設業,1298600,2791.000000
4,オリエンタル白石,1786,建設業,772800,308.000000
...,...,...,...,...,...
449,ＳＣＳＫ,9719,情報・通信業,1138100,2332.500000
450,ベネッセホールディングス,9783,サービス業,765500,1846.500000
451,ヤマダホールディングス,9831,小売業,8603800,428.899994
452,ミスミグループ本社,9962,卸売業,1816100,3096.000000


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2541800,644.200012
1,安藤・間,1719,建設業,1043300,1059.000000
2,コムシスホールディングス,1721,建設業,1298600,2791.000000
3,大成建設,1801,建設業,1535200,4786.000000
4,大林組,1802,建設業,2708300,1181.000000
...,...,...,...,...,...
222,四国電力,9507,電気・ガス業,1336800,989.900024
223,九州電力,9508,電気・ガス業,4093900,911.500000
224,北海道電力,9509,電気・ガス業,1564900,608.200012
225,東京瓦斯,9531,電気・ガス業,2581600,3189.000000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,2541800,644.200012
1,安藤・間,1719,建設業,1043300,1059.000000
2,コムシスホールディングス,1721,建設業,1298600,2791.000000
3,大成建設,1801,建設業,1535200,4786.000000
4,大林組,1802,建設業,2708300,1181.000000
...,...,...,...,...,...
186,四国電力,9507,電気・ガス業,1336800,989.900024
187,九州電力,9508,電気・ガス業,4093900,911.500000
188,北海道電力,9509,電気・ガス業,1564900,608.200012
189,東京瓦斯,9531,電気・ガス業,2581600,3189.000000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,三井住友建設,1821,建設業,690700,368.000000
1,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,7121400,1122.000000
2,リンクアンドモチベーション,2170,サービス業,520800,443.000000
3,明治ホールディングス,2269,食料品,1608800,3242.000000
4,システナ,2317,情報・通信業,1284100,319.000000
...,...,...,...,...,...
64,イーレックス,9517,電気・ガス業,876200,1180.000000
65,レノバ,9519,電気・ガス業,1431300,1575.000000
66,エイチ・アイ・エス,9603,サービス業,3648400,2146.000000
67,ベネッセホールディングス,9783,サービス業,765500,1846.500000


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,エスプール,2471,サービス業,932400,534.000000
1,伊藤園,2593,食料品,637400,4100.000000
2,グリー,3632,情報・通信業,769700,661.000000
3,ｇｕｍｉ,3903,情報・通信業,1729300,684.000000
4,住友ファーマ,4506,医薬品,2449400,649.599976
5,ＪＣＲファーマ,4552,医薬品,671100,1347.000000
6,リソー教育,4714,サービス業,1390600,285.000000
7,楽天グループ,4755,サービス業,43554600,515.700012
8,東邦チタニウム,5727,非鉄金属,1431300,1764.000000
9,アサヒホールディングス,5857,非鉄金属,583500,1946.000000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '2_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,542700,2574.0
1,住石ホールディングス,1514,鉱業,536400,369.0
2,中小企業ホールディングス,1757,建設業,1027200,45.0
3,アジアゲートホールディングス,1783,建設業,1092800,56.0
4,燦キャピタルマネージメント,2134,サービス業,14255500,32.0
...,...,...,...,...,...
102,乾汽船,9308,海運業,215500,1304.0
103,ビート・ホールディングス・リミテッド,9399,情報・通信業,9065759,5.0
104,アゴーラ ホスピタリティー グループ,9704,サービス業,402700,26.0
105,北沢産業,9930,卸売業,445400,417.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,536400,369.000000
1,中小企業ホールディングス,1757,建設業,1027200,45.000000
2,ｎｍｓ ホールディングス,2162,サービス業,342700,460.000000
3,ＣＳＳホールディングス,2304,サービス業,261500,981.000000
4,シダー,2435,サービス業,201500,237.000000
5,ＪＴＰ,2488,サービス業,277400,1049.000000
6,まんだらけ,2652,小売業,246800,1987.000000
7,イメージ ワン,2667,卸売業,1094800,873.000000
8,コメ兵ホールディングス,2780,小売業,224200,4445.000000
9,プロパスト,3236,不動産業,249800,169.000000


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,536400,369.000000
1,ＣＳＳホールディングス,2304,サービス業,261500,981.000000
2,ＪＴＰ,2488,サービス業,277400,1049.000000
3,まんだらけ,2652,小売業,246800,1987.000000
4,イメージ ワン,2667,卸売業,1094800,873.000000
5,コメ兵ホールディングス,2780,小売業,224200,4445.000000
6,プロパスト,3236,不動産業,249800,169.000000
7,プレサンスコーポレーション,3254,不動産業,212000,1951.000000
8,テクノフレックス,3449,金属製品,228200,1096.000000
9,クラウディアホールディングス,3607,繊維製品,467700,692.000000


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '3_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,542700,2574.0
1,アジアゲートホールディングス,1783,建設業,1092800,56.0
2,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1753100,62.0
3,クシム,2345,情報・通信業,1176500,327.0
4,出前館,2484,情報・通信業,724700,427.0
5,ピクセルカンパニーズ,2743,卸売業,5697200,47.0
6,セリア,2782,小売業,259100,2288.0
7,レカム,3323,卸売業,2222800,94.0
8,ブロードバンドタワー,3776,情報・通信業,23511500,147.0
9,リミックスポイント,3825,情報・通信業,635600,241.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,542700,2574.0
1,クシム,2345,情報・通信業,1176500,327.0
2,ピクセルカンパニーズ,2743,卸売業,5697200,47.0
3,セリア,2782,小売業,259100,2288.0
4,リミックスポイント,3825,情報・通信業,635600,241.0
5,神東塗料,4615,化学,311500,129.0
6,メディアリンクス,6659,電気機器,330600,50.0
7,ピクセラ,6731,電気機器,2426900,3.0
8,大黒屋ホールディングス,6993,小売業,1355100,54.0
9,プロルート丸光,8256,卸売業,729100,48.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '4_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,2117600,1076.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,1100200,729.0
2,ソフトフロントホールディングス,2321,情報・通信業,930300,117.0
3,イオレ,2334,サービス業,255800,1926.0
4,メディネット,2370,サービス業,2163800,68.0
...,...,...,...,...,...
117,マイクロ波化学,9227,サービス業,587900,1725.0
118,サンウェルズ,9229,サービス業,891200,3715.0
119,ｉｓｐａｃｅ,9348,サービス業,1736400,1713.0
120,リニューアブル・ジャパン,9522,電気・ガス業,1176600,796.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ソフトフロントホールディングス,2321,情報・通信業,930300,117.0
1,イオレ,2334,サービス業,255800,1926.0
2,海帆,3133,小売業,4890700,612.0
3,ファンデリー,3137,小売業,356200,442.0
4,霞ヶ関キャピタル,3498,不動産業,283300,4580.0
5,ロコンド,3558,小売業,228100,1740.0
6,アドバンスト・メディア,3773,情報・通信業,369500,2065.0
7,Ａｉｍｉｎｇ,3911,情報・通信業,2673400,458.0
8,シェアリングテクノロジー,3989,情報・通信業,854600,745.0
9,スタメン,4019,情報・通信業,359900,1051.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ソフトフロントホールディングス,2321,情報・通信業,930300,117.0
1,イオレ,2334,サービス業,255800,1926.0
2,霞ヶ関キャピタル,3498,不動産業,283300,4580.0
3,ロコンド,3558,小売業,228100,1740.0
4,シェアリングテクノロジー,3989,情報・通信業,854600,745.0
5,スタメン,4019,情報・通信業,359900,1051.0
6,フィーチャ,4052,情報・通信業,926000,1368.0
7,ティアンドエス,4055,情報・通信業,470800,3105.0
8,プレイド,4165,情報・通信業,3505400,869.0
9,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,618900,1419.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '5_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,2117600,1076.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,1100200,729.0
2,メディネット,2370,サービス業,2163800,68.0
3,ディー・ディー・エス,3782,情報・通信業,602500,50.0
4,フィスコ,3807,情報・通信業,225400,120.0
5,グローバルウェイ,3936,情報・通信業,264300,218.0
6,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,262700,344.0
7,ココナラ,4176,情報・通信業,621700,392.0
8,アクリート,4395,情報・通信業,856900,1916.0
9,ミンカブ・ジ・インフォノイド,4436,情報・通信業,544700,1767.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,グローバルウェイ,3936,情報・通信業,264300,218.0
1,リボミック,4591,医薬品,321700,160.0
2,ソレイジア・ファーマ,4597,医薬品,848100,45.0
3,プレミアアンチエイジング,4934,化学,450500,1117.0
4,トレンダーズ,6069,サービス業,1088400,1152.0
5,ＡｐｐＢａｎｋ,6177,サービス業,369400,120.0
6,ＮｅｘＴｏｎｅ,7094,サービス業,282600,2065.0
7,アイドマ・ホールディングス,7373,サービス業,1209900,2744.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,542700,2574.000000
1,住石ホールディングス,1514,鉱業,536400,369.000000
2,中小企業ホールディングス,1757,建設業,1027200,45.000000
3,アジアゲートホールディングス,1783,建設業,1092800,56.000000
4,燦キャピタルマネージメント,2134,サービス業,14255500,32.000000
5,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1753100,62.000000
6,クシム,2345,情報・通信業,1176500,327.000000
7,出前館,2484,情報・通信業,724700,427.000000
8,イメージ ワン,2667,卸売業,1094800,873.000000
9,ピクセルカンパニーズ,2743,卸売業,5697200,47.000000


In [34]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '7_東証S_選抜')

In [35]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,2117600,1076.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,1100200,729.0
2,ソフトフロントホールディングス,2321,情報・通信業,930300,117.0
3,メディネット,2370,サービス業,2163800,68.0
4,ウェッジホールディングス,2388,その他金融業,1055000,109.0
...,...,...,...,...,...
66,マイクロ波化学,9227,サービス業,587900,1725.0
67,サンウェルズ,9229,サービス業,891200,3715.0
68,ｉｓｐａｃｅ,9348,サービス業,1736400,1713.0
69,リニューアブル・ジャパン,9522,電気・ガス業,1176600,796.0


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '8_東証G_選抜')